In [ ]:
##install libraies needed
#?pip
#%pip install -U scikit-learn
##for converting genename, not actually needed
#%pip install mygene
#%pip install -U imbalanced-learn

In [1]:
##import libraries
import os
import mygene
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.metrics import geometric_mean_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import EasyEnsembleClassifier

In [2]:
##move to working dir
os.chdir('/home/atimms/ngs_data/exomes/working/exome_project_20/exome_project_1220/working')
!ls

genelist_ghayda_1120.txt
genelist_kim_1120.txt
genelists_0420.txt
GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct


In [3]:
##load data
gtex_file = 'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct'
gtex_data = pd.read_csv(gtex_file, sep='\t', skiprows=2, index_col=0)
#print(gtex_data.head())
##get list of dbd genes from kim and ghaydas gene list
kim_genes = pd.read_csv('genelist_kim_1120.txt', sep='\t')
ghayda_genes = pd.read_csv('genelist_ghayda_1120.txt', sep='\t')
gg = ghayda_genes.genename.values.tolist()
kg = kim_genes.GENE.values.tolist()
#print(gg)
dbd_genes = list(set(gg + kg))
print(len(dbd_genes), len(gg), len(kg))

896 1392 151


In [4]:
print(gtex_data.shape)
print(gtex_data.describe())

(56200, 55)
       Adipose - Subcutaneous  Adipose - Visceral (Omentum)  Adrenal Gland  \
count            56200.000000                  56200.000000   56200.000000   
mean                16.684563                     16.295418      16.944043   
std                379.706115                    406.450940     587.938639   
min                  0.000000                      0.000000       0.000000   
25%                  0.000000                      0.000000       0.000000   
50%                  0.024909                      0.025098       0.007575   
75%                  2.056895                      1.905715       1.498840   
max              34957.800000                  37804.500000   58547.900000   

       Artery - Aorta  Artery - Coronary  Artery - Tibial       Bladder  \
count    56200.000000       56200.000000     56200.000000  56200.000000   
mean        16.706238          16.191928        16.740760     16.101082   
std        221.293598         260.183566       246.432345   

In [5]:
##add column if gene in dbd gene lists
gtex_data['dbd_gene']=0
gtex_data.loc[gtex_data.Description.isin(dbd_genes),'dbd_gene']='1'
print(gtex_data['dbd_gene'].value_counts())
print(gtex_data.shape)
#print(gtex_data.dtypes)
gtex_data['dbd_gene']=pd.to_numeric(gtex_data['dbd_gene'])
#print(gtex_data.dtypes)

0    55360
1      840
Name: dbd_gene, dtype: int64
(56200, 56)


# so have data set up, so try out different models set ups etc

In [10]:
# calculate precision-recall area under curve
def pr_auc(y_true, probas_pred):
	# calculate precision-recall curve
	p, r, _ = precision_recall_curve(y_true, probas_pred)
	# calculate area under curve
	return auc(r, p)

In [9]:
#Split data into training and test sets
y = gtex_data.dbd_gene
X = gtex_data.drop(['dbd_gene','Description'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123)
X_train.head()

,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Bladder,Brain - Amygdala,Brain - Anterior cingulate cortex (BA24),Brain - Caudate (basal ganglia),...,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Small Intestine - Terminal Ileum,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole Blood
Name,,,,,,,,,,,,,,,,,,,,,
ENSG00000279836.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000279250.1,0.208613,0.114644,0.060242,0.115642,0.063839,0.037143,0.095191,0.00000,0.000000,0.000000,...,0.064861,0.105126,0.000000,0.000000,0.000000,0.264454,0.000000,0.000000,0.000000,0.000000
ENSG00000128266.8,11.784600,11.988600,6.399450,6.245600,5.107930,7.138130,9.838130,23.45160,32.250100,21.085000,...,1.666310,2.002930,2.444320,4.451180,10.494700,10.204600,2.941500,18.123500,4.692020,1.598180
ENSG00000213169.3,0.210841,0.214133,0.146507,0.170044,0.178558,0.236677,0.178300,0.07037,0.060483,0.058378,...,0.348522,0.372033,0.182627,0.181523,0.169496,0.616526,0.249106,0.306844,0.257519,0.110364
ENSG00000266149.1,0.077905,0.063581,0.000000,0.048391,0.000000,0.000000,0.061465,0.00000,0.048001,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.373742,0.185951,0.355927,0.063442,0.000000,0.000000


In [ ]:
##comparing different Random Forest methods -- mlm
'''
y = gtex_data.dbd_gene
X = gtex_data.drop(['dbd_gene','Description'], axis=1)
n_est = 100

# define model
rfc = RandomForestClassifier(n_estimators=n_est)
rfc_bcw = RandomForestClassifier(n_estimators=n_est, class_weight='balanced')
rfc_bscw = RandomForestClassifier(n_estimators=n_est, class_weight='balanced_subsample')
brfc = BalancedRandomForestClassifier(n_estimators=n_est)
ees = EasyEnsembleClassifier(n_estimators=n_est)

# define evaluation procedure
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
cv = 5

# evaluate model
rfc_scores = cross_val_score(rfc, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
rfc_bcw_scores = cross_val_score(rfc_bcw, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
rfc_bscw_scores = cross_val_score(rfc_bscw, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
brfc_scores = cross_val_score(brfc, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
ees_scores = cross_val_score(ees, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)

# summarize performance
print("=== AUC Scores ===")
print(rfc_scores)
print('Mean ROC AUC rfc: %.3f' % np.mean(rfc_scores))
print('\n')
print(rfc_bcw_scores)
print('Mean ROC AUC rfc_bcw: %.3f' % np.mean(rfc_bcw_scores))
print('\n')
print(rfc_bscw_scores)
print('Mean ROC AUC rfc_bscw: %.3f' % np.mean(rfc_bscw_scores))
print('\n')
print(brfc_scores)
print('Mean ROC AUC brfc: %.3f' % np.mean(brfc_scores))
print('\n')
print(ees_scores)
print('Mean ROC AUC ees_scores: %.3f' % np.mean(ees_scores))
'''

In [ ]:
##rf vs balance rf -https://imbalanced-learn.org/stable/auto_examples/ensemble/plot_comparison_ensemble_classifier.html#sphx-glr-auto-examples-ensemble-plot-comparison-ensemble-classifier-py

'''
#Split data into training and test sets
y = gtex_data.dbd_gene
X = gtex_data.drop(['dbd_gene','Description'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123)
y_train.head()
##testing pipeline
my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=10))
cv_scores = cross_val_score(my_pipeline, X_train, y_train, 
                            cv=5,
                            scoring='roc_auc')
#scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)

print("Cross-validation roc_auc: %f" % cv_scores.mean())
##looking a cm etc
rf = RandomForestClassifier(n_estimators=500, random_state=0)
brf = BalancedRandomForestClassifier(n_estimators=500, random_state=0)

rf.fit(X_train, y_train)
brf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_pred_brf = brf.predict(X_test)


print('Random Forest classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_rf),
              geometric_mean_score(y_test, y_pred_rf)))
cm_rf = confusion_matrix(y_test, y_pred_rf)
print(cm_rf)
# fig, ax = plt.subplots(ncols=2)
# plot_confusion_matrix(cm_rf, classes=np.unique(satimage.target), ax=ax[0],
#                        title='Random forest')

print('Balanced Random Forest classifier performance:')
print('Balanced accuracy: {:.2f} - Geometric mean {:.2f}'
      .format(balanced_accuracy_score(y_test, y_pred_brf),
              geometric_mean_score(y_test, y_pred_brf)))
cm_brf = confusion_matrix(y_test, y_pred_brf)
print(cm_brf)
# plot_confusion_matrix(cm_brf, classes=np.unique(satimage.target), ax=ax[1],
#                       title='Balanced random forest')
'''